# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4929, uptime 0:00:34
mini_beamline                    RUNNING   pid 4930, uptime 0:00:34
random_walk                      RUNNING   pid 4931, uptime 0:00:34
random_walk_horiz                RUNNING   pid 4932, uptime 0:00:34
random_walk_vert                 RUNNING   pid 4945, uptime 0:00:32
simple                           RUNNING   pid 4934, uptime 0:00:34
thermo_sim                       RUNNING   pid 4935, uptime 0:00:34
trigger_with_pc                  RUNNING   pid 4936, uptime 0:00:34


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-06-01 20:19:24
Persistent Unique Scan ID: '8e6482ba-f4ff-4a80-a007-c33170985fa1'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:19:24.2 |     -1.000 |      95717 |
|         2 | 20:19:24.2 |     -0.500 |      97701 |
|         3 | 20:19:24.3 |      0.000 |      98752 |
|         4 | 20:19:24.3 |      0.500 |      98501 |
|         5 | 20:19:24.3 |      1.000 |      97119 |


+-----------+------------+------------+------------+
generator scan ['8e6482ba'] (scan num: 3)





('8e6482ba-f4ff-4a80-a007-c33170985fa1',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/8e6482ba-f4ff-4a80-a007-c33170985fa1-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-06-01 20:19:24
Persistent Unique Scan ID: 'fe7b219b-c1c6-4d3b-9a3d-805036320ba9'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:19:24.8 |          -1 | 1102413235 |


|         2 | 20:19:25.0 |          -0 | 1112003356 |


|         3 | 20:19:25.1 |           0 | 1122039592 |


|         4 | 20:19:25.2 |           0 | 1125713088 |


|         5 | 20:19:25.3 |           1 | 1128831242 |


+-----------+------------+-------------+------------+
generator scan ['fe7b219b'] (scan num: 4)





('fe7b219b-c1c6-4d3b-9a3d-805036320ba9',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-06-01 20:19:25
Persistent Unique Scan ID: 'ef55f875-003c-4d72-a6ce-7edfc1b253ff'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:19:25.9 | 1128731290 |


|         2 | 20:19:26.8 | 1076036786 |


|         3 | 20:19:27.8 | 1057009934 |


...

|        28 | 20:19:52.8 | 1111373771 |


|        29 | 20:19:53.8 | 1126771976 |


|        30 | 20:19:54.8 | 1073083016 |


+-----------+------------+------------+
generator count ['ef55f875'] (scan num: 5)





('ef55f875-003c-4d72-a6ce-7edfc1b253ff',)